# Throughtput Benchmarking  Seldon-Core on GCP Kubernetes

The notebook will provide a benchmarking of seldon-core for maximum throughput test. We will run a stub model and test using REST and gRPC predictions. This will provide a maximum theoretical throughtput for model deployment in the given infrastructure scenario:
  
   * 1 replica of the model running on n1-standard-16 GCP node
   
For a real model the throughput would be less. Future benchmarks will test realistic models scenarios.


## Create Cluster

Create a cluster of 4 nodes of machine type n1-standard-16 

```bash
PROJECT=seldon-core-benchmarking
ZONE=europe-west1-b
gcloud beta container --project "${PROJECT}" clusters create "loadtest" \
    --zone "${ZONE}" \
    --username "admin" \
    --cluster-version "1.9.3-gke.0" \
    --machine-type "n1-standard-16" \
    --image-type "COS" \
    --disk-size "100" \
    --num-nodes "4" \
    --network "default" \
    --enable-cloud-logging \
    --enable-cloud-monitoring \
    --subnetwork "default"
```

## Install helm

In [2]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


## Start Seldon-Core CRD

In [4]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Mar  7 15:14:11 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s


NOTES:
NOTES: TODO




## Cordon off loadtest nodes

In [5]:
!kubectl get nodes

NAME                                      STATUS    ROLES     AGE       VERSION
gke-loadtest-default-pool-e2e99314-7zb5   Ready     <none>    1m        v1.9.3-gke.0
gke-loadtest-default-pool-e2e99314-gbjx   Ready     <none>    1m        v1.9.3-gke.0
gke-loadtest-default-pool-e2e99314-hcvx   Ready     <none>    1m        v1.9.3-gke.0
gke-loadtest-default-pool-e2e99314-tb2p   Ready     <none>    1m        v1.9.3-gke.0


We cordon off first 3 nodes so seldon-core and the model will not be deployed on the 1 remaining node.

In [6]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node "gke-loadtest-default-pool-e2e99314-7zb5" cordoned
node "gke-loadtest-default-pool-e2e99314-gbjx" cordoned
node "gke-loadtest-default-pool-e2e99314-hcvx" cordoned


Label the nodes so they can be used by locust.

In [7]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}') role=locust

node "gke-loadtest-default-pool-e2e99314-7zb5" labeled
node "gke-loadtest-default-pool-e2e99314-gbjx" labeled
node "gke-loadtest-default-pool-e2e99314-hcvx" labeled


## Start seldon-core

In [8]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set cluster_manager.rbac=true \
        --set apife.enabled=true \
        --set engine.image.tag=0.1.6_SNAPSHOT_loadtest \
        --set cluster_manager.image.tag=0.1.6_SNAPSHOT_loadtest
        

NAME:   seldon-core
LAST DEPLOYED: Wed Mar  7 15:15:11 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
redis-df886d999-rl5l8  0/1    ContainerCreating  0         0s

==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-apiserver        1        0        0           0          1s
seldon-cluster-manager  1        0        0           0          1s
redis                   1        1        1           0          1s

==> v1/Service
NAME              TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)                        AGE
seldon-apiserver  NodePort   10.3.246.186  <none>       8080:31873/TCP,5000:30398/TCP  1s
redis             ClusterIP  10.3.240.223  <none>       6379/TCP                       0s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s


NOTES:
NOTES: TODO




Wait for seldon-core to start

In [11]:
!kubectl get pods -o wide

NAME                                      READY     STATUS    RESTARTS   AGE       IP         NODE
redis-df886d999-rl5l8                     1/1       Running   0          31s       10.0.2.5   gke-loadtest-default-pool-e2e99314-tb2p
seldon-apiserver-64ccd4c5f4-xcclp         1/1       Running   0          31s       10.0.2.7   gke-loadtest-default-pool-e2e99314-tb2p
seldon-cluster-manager-68c8c6b5bf-tbc5b   1/1       Running   0          31s       10.0.2.6   gke-loadtest-default-pool-e2e99314-tb2p


## Create Stub Deployment

In [12]:
!pygmentize resources/loadtest_simple_model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-core-loadtest"
    },
    "spec": {
        "annotations": {
            "project_name": "loadtest",
            "deployment_version": "v1"
        },
        "name": "loadtest",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                        ],
                        "terminationGracePeriodSeconds": 20
                    }
                },
                "graph": {
                    "children": [],
                    "name": "stub",
                    "endpoint": {
			"type" : "REST"
		    },
                    "type": "MODEL",
		    "implementation": "SIMPLE_MODEL"
                },
                "name": "load

In [13]:
!kubectl apply -f resources/loadtest_simple_model.json

seldondeployment "seldon-core-loadtest" created


Wait for deployment to be running.

In [15]:
!kubectl get seldondeployments seldon-core-loadtest -o jsonpath='{.status}'

map[predictorStatus:[map[name:loadtest-loadtest replicas:1 replicasAvailable:1]]]

## Run benchmark

Uncorden the first 3 nodes so they can be used to schedule locust

In [16]:
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node "gke-loadtest-default-pool-e2e99314-7zb5" uncordoned
node "gke-loadtest-default-pool-e2e99314-gbjx" uncordoned
node "gke-loadtest-default-pool-e2e99314-hcvx" uncordoned


## gRPC
Start locust load test for gRPC

In [17]:
!helm install ../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=loadtest:5001 \
    --set locust.script=predict_grpc_locust.py \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=64 

NAME:   loadtest
LAST DEPLOYED: Wed Mar  7 15:17:44 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME             TYPE      CLUSTER-IP    EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.3.253.118  <none>       5557:30970/TCP,5558:30185/TCP,8089:30505/TCP  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-2kc26   0/1    Pending            0         0s
locust-slave-1-2ngks   0/1    Pending            0         1s
locust-slave-1-4kxg4   0/1    Pending            0         1s
locust-slave-1-5h67f   0/1    Pending            0         1s
locust-slave-1-62c7w   0/1    Pending            0         1s
locust-slave-1-68skx   0/1    ContainerCreating  0         1s
locust-slave-1-6qnwx   0/1    ContainerCreating  0         1s
locust-slave-1-9d4vm   0/1    Pending            0         1s
locust-slave-1-b9fgb   0/1    Pending            0         1s
locust-slave-1-bf7nq   0/1    P

To download stats use 

```bash
if [ "$#" -ne 2 ]; then
    echo "Illegal number of parameters: <experiment> <rest|grpc>"
fi

EXPERIMENT=$1
TYPE=$2

MASTER=`kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}'`

kubectl cp ${MASTER}:stats_distribution.csv ${EXPERIMENT}_${TYPE}_stats_distribution.csv
kubectl cp ${MASTER}:stats_requests.csv ${EXPERIMENT}_${TYPE}_stats_requests.csv
```

You can get live stats by viewing the logs of the locust master

In [21]:
!kubectl logs $(kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}') --tail=10

 grpc loadtest:5001                                            875872     0(0.00%)       1       0    5020  |       1 26458.80
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                         875872     0(0.00%)                                   26458.80

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s
--------------------------------------------------------------------------------------------------------------------------------------------
 grpc loadtest:5001                                            917872     0(0.00%)       1       0    5020  |       1 25319.50
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                         917872 

In [22]:
!helm delete loadtest --purge

release "loadtest" deleted


## REST 
Run REST benchmark

In [23]:
!helm install ../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://loadtest:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=64

NAME:   loadtest
LAST DEPLOYED: Wed Mar  7 15:20:13 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   64       0        0      1s
locust-master-1  1        1        0      1s

==> v1/Service
NAME             TYPE      CLUSTER-IP    EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.3.243.232  <none>       5557:31799/TCP,5558:32699/TCP,8089:30737/TCP  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-29sc7   0/1    Pending            0         1s
locust-slave-1-4h2jz   0/1    ContainerCreating  0         1s
locust-slave-1-545xr   0/1    Pending            0         1s
locust-slave-1-55vbz   0/1    Pending            0         1s
locust-slave-1-59lff   0/1    Pending            0         1s
locust-slave-1-dtcvq   0/1    Pending            0         1s
locust-slave-1-dzh8c   0/1    Pending            0 

Get stats as per gRPC and/or monitor

In [24]:
!kubectl logs $(kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}') --tail=10

 POST predictions                                              250653     0(0.00%)       5       2    5011  |       4 11907.80
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                         250653     0(0.00%)                                   11907.80

 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s
--------------------------------------------------------------------------------------------------------------------------------------------
 POST predictions                                              272674     0(0.00%)       5       2    5011  |       4 11785.90
--------------------------------------------------------------------------------------------------------------------------------------------
 Total                                                         272674 

In [25]:
!helm delete loadtest --purge

release "loadtest" deleted


In [26]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

node "gke-loadtest-default-pool-e2e99314-7zb5" cordoned
node "gke-loadtest-default-pool-e2e99314-gbjx" cordoned
node "gke-loadtest-default-pool-e2e99314-hcvx" cordoned


## Tear Down

In [27]:
!kubectl delete -f resources/loadtest_simple_model.json

seldondeployment "seldon-core-loadtest" deleted


In [28]:
!helm delete seldon-core --purge

release "seldon-core" deleted


In [29]:
!helm delete seldon-core-crd --purge

release "seldon-core-crd" deleted
